In [10]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras as K
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from dataset import parse_tree_generator, print_stronghold_tree
from anytree import Node, RenderTree, Walker
from anytree.search import find_by_attr
import random

In [17]:
df = pd.read_csv('100k_dataset_rnn.csv', delimiter=' ')

In [18]:
df.head(9)

,room,parent_room,parent_exit,child_room_1,child_room_2,child_room_3,child_room_4,child_room_5,exit
0,LeftTurn,ChestCorridor,1,RightTurn,None,None,None,None,0
1,ChestCorridor,RightTurn,1,LeftTurn,None,None,None,None,0
2,RightTurn,Corridor,3,ChestCorridor,None,None,None,None,0
3,Corridor,FiveWayCrossing,1,RightTurn,ChestCorridor,RightTurn,None,None,0
4,FiveWayCrossing,Start,1,Corridor,LeftTurn,None,Corridor,Stairs,2
5,LeftTurn,FiveWayCrossing,2,Corridor,None,None,None,None,1
6,Corridor,LeftTurn,1,None,Corridor,None,None,None,2
7,Corridor,Corridor,2,SquareRoom,None,None,None,None,1
8,SquareRoom,Corridor,1,SpiralStaircase,PortalRoom,LeftTurn,None,None,2


In [19]:
# one-hot encode
cols = [
        'room',
        'parent_room',
        'parent_exit',
        'child_room_1',
        'child_room_2',
        'child_room_3',
        'child_room_4',
        'child_room_5']
onehot = ColumnTransformer([("one-hot", OneHotEncoder(), cols)], remainder='passthrough')
onehot.fit(df)
df_onehot = pd.DataFrame(onehot.transform(df).toarray(), index=df.index, columns=pd.Index(onehot.get_feature_names()))

In [22]:
df_onehot.head()

,one-hot__x0_ChestCorridor,one-hot__x0_Corridor,one-hot__x0_FiveWayCrossing,one-hot__x0_LeftTurn,one-hot__x0_PrisonHall,one-hot__x0_RightTurn,one-hot__x0_SpiralStaircase,one-hot__x0_SquareRoom,one-hot__x0_Stairs,one-hot__x1_ChestCorridor,...,one-hot__x7_Library,one-hot__x7_None,one-hot__x7_PortalRoom,one-hot__x7_PrisonHall,one-hot__x7_RightTurn,one-hot__x7_SmallCorridor,one-hot__x7_SpiralStaircase,one-hot__x7_SquareRoom,one-hot__x7_Stairs,exit
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0


In [37]:
X_train, X_test, y_train, y_test = train_test_split(
        df_onehot.drop('exit', axis=1),
        df_onehot['exit'],
        test_size=0.1,
        random_state=1337)

In [30]:
print("X_train:", X_train.shape)
print("y_train:", y_train.shape)
print("X_test:", X_test.shape)
print("y_test:", y_test.shape)

X_train: (633797, 89)
y_train: (633797,)
X_test: (70422, 89)
y_test: (70422,)


In [50]:
X_train_np = X_train.to_numpy()
X_train_np = X_train_np.reshape(X_train_np.shape[0], 1, X_train_np.shape[1])
y_train_np = y_train.to_numpy()

In [51]:
model = K.Sequential()
model.add(K.layers.LSTM(128, input_shape=(None, X_train_np.shape[2])))
model.add(K.layers.Dense(6, activation='softmax'))
model.compile(
        optimizer='adam',
        loss=K.losses.SparseCategoricalCrossentropy(),
        metrics=['accuracy'])
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 128)               111616    
_________________________________________________________________
dense_8 (Dense)              (None, 6)                 774       
Total params: 112,390
Trainable params: 112,390
Non-trainable params: 0
_________________________________________________________________


In [53]:
model.fit(X_train_np, y_train_np, epochs=10)

Epoch 1/10
19807/19807 [==============================] - 21s 1ms/step - loss: 0.6903 - accuracy: 0.6650
Epoch 2/10
19807/19807 [==============================] - 21s 1ms/step - loss: 0.6874 - accuracy: 0.6657
Epoch 3/10
19807/19807 [==============================] - 21s 1ms/step - loss: 0.6855 - accuracy: 0.6671
Epoch 4/10
19807/19807 [==============================] - 21s 1ms/step - loss: 0.6842 - accuracy: 0.6678
Epoch 5/10
19807/19807 [==============================] - 21s 1ms/step - loss: 0.6831 - accuracy: 0.6684
Epoch 6/10
19807/19807 [==============================] - 21s 1ms/step - loss: 0.6820 - accuracy: 0.6690
Epoch 7/10
19807/19807 [==============================] - 21s 1ms/step - loss: 0.6809 - accuracy: 0.6692
Epoch 8/10
19807/19807 [==============================] - 21s 1ms/step - loss: 0.6799 - accuracy: 0.6704
Epoch 9/10
19807/19807 [==============================] - 21s 1ms/step - loss: 0.6790 - accuracy: 0.6709
Epoch 10/10
19807/19807 [==============================

In [54]:
model.save("rnn_1.keras")

In [57]:
X_test_np = X_test.to_numpy()
X_test_np = X_test_np.reshape(X_test_np.shape[0], 1, X_test_np.shape[1])
y_test_np = y_test.to_numpy()

In [58]:
model.evaluate(X_test_np, y_test_np)

2201/2201 [==============================] - 2s 676us/step - loss: 0.6861 - accuracy: 0.6672


[0.6860541701316833, 0.6672488451004028]